In [1]:
import pickle
import numpy as np
import pandas as pd

### Load the saved data that has been cleaned and performed feature renormalization

In [2]:
df = pickle.load(open('cleaned_data', 'rb'))

In [3]:
from sklearn.preprocessing import LabelEncoder

In [4]:
train_users_path = './DAL_HW5/train_users_2.csv'
submission_path = './DAL_HW5/sample_submission_NDF.csv'
df_train = pd.read_csv(train_users_path)
df_submission = pd.read_csv(submission_path)
target = df_train['country_destination']

In [5]:
piv_train = len(target) #Marker to split df_all into train + test
vals = df.values
le = LabelEncoder()

X = vals[:piv_train]
y = le.fit_transform(target.values)
X_test = vals[piv_train:]
print('Shape X = %s, Shape X_test = %s, Shape y = %s'%(X.shape, X_test.shape, y.shape))

Shape X = (213451, 661), Shape X_test = (62096, 661), Shape y = (213451,)


## A Simple Logistic Regression

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
clf_log = LogisticRegression(class_weight='balanced', 
                              random_state = 1, 
                              solver = 'sag', 
                              multi_class = 'multinomial',
                              verbose = 3,
                              max_iter = 200, 
                              )

In [ ]:
clf_log.fit(X, y)

In [ ]:
y_pred = le.inverse_transform(clf_log.predict(X_test))

## Xgboost Method

In [11]:
import xgboost as xgb

In [12]:
param = {'max_depth':4, 'eta':1, 'objective':'multi:softmax', 'num_class' : 12, 'eval_metric' : 'logloss', 
         'num_rounds': 5}
dtrain = xgb.DMatrix(X, label=y)

In [14]:
bst = xgb.train(param, dtrain)

In [16]:
dtest = xgb.DMatrix(X_test)

In [19]:
out_label = bst.predict(dtest)

In [22]:
out_label = out_label.astype(int)

In [23]:
y_pred = le.inverse_transform(out_label)

## Neural Network without convolution nor pooling

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, MaxPooling1D, Convolution1D

Using TensorFlow backend.


In [15]:
from keras.utils.np_utils import to_categorical

In [34]:
input_dim = 661
hidden_layer1 = 2000
hidden_layer2 = 1000
hidden_layer3 = 400
hidden_layer4 = 100
output_dim = 12
nb_filter = 250
filter_length = 3

In [35]:
model = Sequential([
    Dense(output_dim = hidden_layer1, input_dim = input_dim),
    Activation('relu'),
    Dropout(0.25),
        
    Dense(output_dim = hidden_layer2, input_dim = hidden_layer1),
    Activation('relu'),
    Dropout(0.25),
    
    Dense(output_dim = hidden_layer3, input_dim = hidden_layer2),
    Activation('relu'),
    
    Dense(output_dim = hidden_layer4, input_dim = hidden_layer3),
    Activation('relu'),
    
    Dense(output_dim= output_dim, input_dim = hidden_layer4),
    Activation('softmax')
])

In [36]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [37]:
n_classes = 12
y_train = to_categorical(y, n_classes)

In [38]:
model.fit(X, y_train, validation_split = 0.2, nb_epoch = 10, batch_size = 320)

Train on 170760 samples, validate on 42691 samples
Epoch 1/1
170760/170760 [==============================] - 159s - loss: 11.3068 - acc: 0.2985 - val_loss: 11.8355 - val_acc: 0.2657


In [42]:
y_pred = model.predict(X_test)

## Submission

In [ ]:
df_submission.country = y_pred

In [ ]:
df_submission.to_csv(submission_path)